# Projet Julia : problème d'échange de reins avec un Branch and Price

Roméo Legoupil, Matthieu Roux, Marie-Charlotte Fougère--Ballé, Marc Le Moing

## Introduction

La maladie rénale chronique est une des maladies graces qui menacent la vie de nombreuses personnes. Cette maladie est la onzième cause de décès dans le monde, environ 2,17% des décès annuels. Il existe deux traitements pour cette maladie : 
- la dialyse : elle est plus facilement accessible mais elle nécessite de nombreuses visites à l'hôpital et des dépenses considérables. Cela réduit donc la qualité de vie du patient.
- la transplantation d'organe (de rein) : lorsqu'elle est réalisée avec succès, cela permet au patient de poursuivre sa vie sans aucun problème de santé.
Le traitement privilégié est donc la transplantation d'organe.

Normalement, ces transplantations sont réalisées à partir de donneurs décédés. Or, le nombre de patients  en attente d'un organe dépasse le nombre d'organes disponibles. Ainsi, il est maintenant possible de recevoir le rein d'un donneur vivant. Une transplantation rénale avec donneur vivant est réalisée entre un patient et un donner disposé à lui donner un de ses reins. Des tests de compatibilité doivent aussi être effectués entre le patient et le donneur avant que l'opération ait lieue. Lorsque le patient et le donneur sont incompatibles, les programmes d'échange de reins sont une solution alternative.

Un programme d'échange de reins est un système qui contient un ensemble de paires patient-donneur incompatibles. On peut apparier un patient avec le donneur vivant d'un autre patient avec lequel il est compatible. Les cycles d'échanges peuvent donc être construits de telle sorte que le donneur de chaque paire du cycle donne son rein au patient de la paire suivante et que le donneur de la dernière paire donne son rein au patient de la première paire. Les cycles longs ne sont pas souhaitables en raison de contraintes éthiques et logistiques. Ainsi, une longueur maximale de cycle est très souvent imposée.

Dans ce projet, nous allons donc trouver la meilleure façon de créer des cycles d'échange de reins de manière à maximiser le bien-être commun tout en respectant une longueur maximale de cycles d'échange. Ce problème est appelé problème d'échange de reins.

## Notations

Le problème d'échange de reins, que l'on note KEP par la suite, peut être représenté par un simple graphe orienté G=(V,A). V représente les paires patient-donneur $(P_i,D_i)$ et A représente la compatibilité entre les paires. Ainsi, $(i,j) \in A$ si le donneur $D_i$ et compatible avec le patient $P_j$.

Un score de priorité ou d'utilité est attribué à chaque arc $(i,j)$. L'arc prendre la valeur 1 si toutes les greffes sont considérées comme égales. L'objectif est de maximiser le nombre de greffes réalisées. L'utilité de l'arc $(i,j)$ est notée par $w_{ij}$.

Soit $L$ la longueur maximale d'un cycle. On note $C_L$ l'ensemble des cycles de G tel que $|c| \le L$ pour $c \in C_L$. $C_L$ représente donc l'ensemble des cycles de G de longueur inférieure à $L$. On définit $C_L(i)$ l'ensemble des cycles qui contiennent le noeuds $i \in V$ et $w_c=\sum_{(i,j) \in c} w_{ij}$ pour $c \in C_L$.

On a la variable $z_c$ pour $c \in C_L(i)$ qui vaut :
- 1 si le cycle $c$ est choisi,
- 0 sinon.

Voici une formulation du problème, appellée la formulation par cycles :

max $\sum_{c \in C_L} w_c z_c$

tel que :
- $\sum_{c \in C_L(i)} z_c \le 1, \forall i \in V$, (chaque paire est dans au plus un cycle)
- $z \in \{0,1\}^{|C_L|}$.

Comme cette formulation a un grand nombre de variables, le but du projet est d'obtenir une solution en utilisant l'algorithme du Branch and Price vu en cours. Pour cela, nous allons trouver le problème maitre, les sous problèmes et comment générer une colonne afin de générer l'algorithme de la génération de colonnes. De plus, pour implémenter le Branch and Price, nous allons devoir trouver et éliminer les solutions fractionnaires ainsi que trouver la façon de brancher.

## Importations et lecture des données

### Importation des packages nécessaires

On importe tous les packages nécessaires pour notre code.

In [3]:
using JuMP 
using Gurobi
using DelimitedFiles
using NBInclude
const GUROBI_ENV = Gurobi.Env()
const ϵ = 0.00001


Set parameter Username
Academic license - for non-commercial use only - expires 2022-07-04


1.0e-5

In [2]:
using DelimitedFiles
using Graphs, GraphPlot#, Compose, LightGraphs, PyPlot

### Importation des fichiers code nécessaires

On importe maintenant les fichiers code nécessaires pour notre projet. En effet, pour mieux organiser notre projet, nous avons séparé notre code en différents fichiers. Chaque fichier permet de coder une partie de notre projet. 

In [2]:
@nbinclude("dataparser.ipynb")
@nbinclude("master.ipynb")
@nbinclude("MIP.ipynb")
@nbinclude("node.ipynb")
@nbinclude("subproblem.ipynb")
@nbinclude("typedef.ipynb")
@nbinclude("branch_and_price.ipynb")

LoadError: LoadError: UndefVarError: @nbinclude not defined
in expression starting at In[2]:1

### Lecture des données

In [4]:
data_folder = string(@__DIR__,"/data")
instance_name = "MD-00001-00000010"
filename = joinpath(data_folder, join([instance_name]))
global new_inst = Instance(filename, 3, 0)

Instance({16, 47} directed simple Int64 graph, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.0 0.0 … 0.0 1.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], Int64[], 16, 0, 3, 0, true)

In [1]:
x = [0, 10, 20, 30, 40, 40, 40, 40, 40, 30, 20, 10, 0, 0, 0, 0]
y = [0, 0, 0, 0, 0, 10, 20, 30, 40, 40, 40, 40, 40, 30, 20, 10]

nodelabel = new_inst.pairs
gplot(g, nodelabel=nodelabel, x, y)

LoadError: UndefVarError: new_inst not defined

In [6]:
g_optim=SimpleDiGraph
for 

16-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15
 16

### Résolution du MIP

Dans cette partie, nous allons résoudre le problème d'échange de reins sur les données que l'on stocke dans la variable $instance\_name$. Pour cela, on appelle la fonction que l'on a créée dans un fichier code séparé.

In [14]:
result = @timed solve_MIP()

Set parameter Username
Academic license - for non-commercial use only - expires 2022-07-04


(value = (4.0, [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]), time = 13.6793017, bytes = 1290064618, gctime = 0.6600094, gcstats = Base.GC_Diff(1290064618, 1, 0, 22009451, 5631, 0, 660009400, 17, 0))

In [28]:
println("La solution optimale est : ",result[1][1])
println("La matrice des solutions est : \n")
for i in 1:nv(new_inst.graph)
    println("ligne ",i," : ", result[1][2][i,:], "\n")
end

La solution optimale est : 4.0
La matrice des solutions est : 

ligne 1 : [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

ligne 2 : [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

ligne 3 : [0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]

ligne 4 : [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

ligne 5 : [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

ligne 6 : [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

ligne 7 : [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

ligne 8 : [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

ligne 9 : [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

ligne 10 : [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

ligne 11 : [0.

On a un échange entre le 3 et le 14 ainsi qu'entre le 6 et le 4. Il est étonnant que le cycle 4-6-5 n'ait pas été pris.

## Branch and price
### Génération de colonne

Afin d'éviter la symétrie, on raisonne en cycle désagrégé. 

Soit L, le nombre maximum de cycle (on peut le considérer égal au nombre de sommet)
Soit $1\leq l \leq L$

- $V^l=\{i\in V, i \geq l, d^l_{il}+d^l_{li} \leq k\}$, l'ensemble des sommets dont le numéro est supérieur à $l$, et dont la distance aller-retour de $i$ à $l$ est inférieure à k.
- $A^l=\{(i,j)\in A, d^l_{li}+d^l_{jl}+1 \leq k \}$, l'ensemble des arêtes reliant les sommets de $V^l$ et permettant de former un cycle passant par i et j de taille inférieure à k.

On note ainsi le sous graphe $G^l=(V^l,A^l)$

On note également $C^l$, l'ensemble des cycles du sous-graphe $G^l$, qui ont une taille inférieure à k et qui inclue le sommet l.

On pose $y^{lc}=1$ si le cycle $c$ de $C^l$ est choisi, $y^{lc}=0$ sinon.

$V^l(c)$ est l'ensemble des sommets du cycle $c \in C^l$, $|c|^l=|V^l(c)|$ est la taille du cycle c (car les arêtes ont un poids de 1).


En posant le sous-ensemble $C_L' \subseteq C_L$ 

Le problème maître restreint est :

\begin{align}
&max \sum_{l \in  \mathcal{L}} \sum_{c \in C_L'} |c|^l y^{lc} \\
&tel que :\\
& \sum_{l \in \mathcal{L}} \sum_{c \in C^l:i \in V^l(c)} y^{lc} \le 1, \forall i \in V \\
& y^{lc} \in \{0,1\}, \forall l \in \mathcal{L}, \forall c \in C^l \\
\end{align}

On note $y^{lc}=1$ si le cycle $c$ est choisi dans l'ensemble des cycles $l$.


Soit $\overline{x}^{lc}_{ij} = 1$ si on prend l'arc (i,j) dans le cycle $c$ avec l'index $l$. On peut réécrire le problème maître comme :

\begin{align}
&max \sum_{l \in  \mathcal{L}} \sum_{c \in C_L'} \sum_{(i,j) \in A^l} x^{lc}_{ij} y^{lc} \\
&tel que :\\
& \sum_{l \in \mathcal{L}} \sum_{c \in C^l:i \in V^l(c)} \sum_{(i,j) \in A^l} x^{lc}_{ij}  y^{lc} \le 1, \forall i \in V  \ \ \ \ [\lambda_i \geq 0] \\
& y^{lc} \in [0,1], \forall l \in \mathcal{L}, \forall c \in C^l \\
\end{align}

Afin de générer des colonnes, il faut résoudre les $L$ sous problèmes corespondant à chaque sous-graphe $G^l$.



Soit $x^l_{ij} = \sum_{c\in C^L}x^{lc}_{ij}y^{lc}$

Le sous problème $Sp(l)$ pour le sommet l est le suivant : 

 
\begin{align}
&\underset{c\in C_L}{min}  \sum_{(i,j)\in A^l}x_{ij}^l - \sum_{j \in V^l} \lambda_j \sum_{(i,j)\in A^l}x_{ij}^l \\
& t.q \\
& \sum_{j:(j,i)\in A^l} x_{ji}^l = \sum_{j:(i,j)\in A^l}x_{ji}^l,  \forall i \in V^l \ \ (un \ seul \ échange d\'organe \ entre \ 2 \ personnes \\
& \sum_{i:(i,j)\in A^l}x_{ij}^l \leq 1,  \forall j \in V^l\\
& \sum_{i:(i,j)\in A^l}x_{il}^l = 1\\
& \sum_{(i,j)\in A^l}x_{ij}^l \leq L ~ ( la \ taille \ du \ cycle \ est \ inférieur \ à \ L)\\
\end{align}

### Pricing
Pour le branchement, on considère les contraintes comme facile. Pour éliminer les cycles fractionnaires, nos contraintes de branchement pour le sous-problèmes $l$ sont :

Soit $(i_1,j_1) \in A^l$, $i_1 \ne j_1$, 

Pour le noeud gauche : $x^l_{i_1j_1} \leq 0$, l'ensemble réalisable du sous-problème est $\{x^l_{ij} \in \{0,1\}, x^l_{i_1j_1} \geq 0, (i,j) \in A^l \}$

Pour le noeud droite : $x^l_{i_1j_1} \geq 1$, l'ensemble réalisable du sous-problème est $\{x^l_{ij} \in \{0,1\}, x^l_{i_1j_1} \leq 0, (i,j) \in A^l \}$

On atteint l'optimalité quand tous les couts réduits des sous-problèmes sont négatifs, c'est à dire :

$$ 
\forall l \in L, \ \sum_{(i,j)\in A^l}x_{ij}^l - \sum_{j \in V^l} \lambda_j \sum_{(i,j)\in A^l}x_{ij}^l
$$


In [25]:
println(column_pool)

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]]


Commment on fait pour passer au noeud suivant, avoir un bazar qui rend une vrai borne inf

In [24]:
result = @timed solve_BP()

Liste actuelle des noeuds à traiter:[1]
 Noeud traité : 1 
Variables mises à 0 sont :Tuple[]
Variables mises à 1 sont :Tuple[]
node_pool super [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

(value = nothing, time = 0.6005027, bytes = 30299135, gctime = 0.0, gcstats = Base.GC_Diff(30299135, 0, 0, 500410, 192, 0, 0, 0, 0))

In [23]:
@nbinclude("dataparser.ipynb")
@nbinclude("master.ipynb")
@nbinclude("MIP.ipynb")
@nbinclude("node.ipynb")
@nbinclude("subproblem.ipynb")
@nbinclude("typedef.ipynb")
@nbinclude("branch_and_price.ipynb")

Nombre de paires : 16
Nombre d'altruistes : 0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
Edge 1 => 3 -- has 1 as its origin and 3 as its destination
Edge 1 => 16 -- has 1 as its origin and 16 as its destination
Edge 2 => 4 -- has 2 as its origin and 4 as its destination
Edge 2 => 14 -- has 2 as its origin and 14 as its destination
Edge 3 => 4 -- has 3 as its origin and 4 as its destination
Edge 3 => 14 -- has 3 as its origin and 14 as its destination
Edge 4 => 6 -- has 4 as its origin and 6 as its destination
Edge 4 => 13 -- has 4 as its origin and 13 as its destination
Edge 4 => 16 -- has 4 as its origin and 16 as its destination
Edge 5 => 1 -- has 5 as its origin and 1 as its destination
Edge 5 => 2 -- has 5 as its origin and 2 as its destination
Edge 5 => 4 -- has 5 as its origin and 4 as its destination
Edge 5 => 6 -- has 5 as its origin and 6 as its destination
Edge 5 => 8 -- has 5 as its origin and 8 as its destination
Edge 5 => 10 -- has 5 as its origin and 10 as its destination
Edg

LoadError: LoadError: syntax: incomplete: "function" at C:\Users\matth\Documents\4GM\Projet-PMAA\node.ipynb:In[7]:1 requires end
in expression starting at C:\Users\matth\Documents\4GM\Projet-PMAA\node.ipynb:In[7]:1

In [76]:
function mindist(dist, sptset,graph)
    min = Inf  # Initialize minimum distance for next node
    minindex = 0
    # Search smallest value nearest vertex not in the
    # shortest path tree
    for i in 1:size(graph)[1]
        if dist[i] < min && sptset[i] == false
            min = dist[i]
            minindex = i
        end
    end
    return minindex
end

mindist (generic function with 2 methods)

In [77]:
function dijkstra(graph, initial_node)
    TreeSet = [false for i in 1:size(graph)[1]] # step 1
    dist = [Inf for i in 1:size(graph)[1]] # step 2
    dist[initial_node] = 0

    for i in 1:size(graph)[1]

        # Pick the minimum distance vertex from
        # the set of vertices not yet processed.
        x = mindist(dist,TreeSet,graph) # step 3
        
        if x!=0
            # step 3 -> relaxation procedure
            # Update dist value of the adjacent vertices
            # of the picked vertex only if the current
            # distance is greater than new distance and
            # the vertex in not in the shortest path tree
            for j in 1:size(graph)[1]
                if graph[x,j] > 0 && TreeSet[j] == false && dist[j] > dist[x] + graph[x,j]
                    dist[j] = dist[x] + graph[x,j]
                end
            end

            # Put the minimum distance vertex in the
            # shortest path tree
            TreeSet[x] = true # step 4
        end
    end
    listeb=[j for (i , j) in enumerate(dist)]
    return listeb
end

dijkstra (generic function with 1 method)

In [78]:
dijkstra((new_inst.edge_weight),1)

16-element Vector{Float64}:
  0.0
 Inf
  1.0
  2.0
 Inf
  3.0
 Inf
 Inf
 Inf
 Inf
 Inf
 Inf
  3.0
  2.0
  3.0
  1.0